# Save and Load Tensorflow model

In [16]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.9.1


[Save and Load Tensorflow model](https://www.tensorflow.org/tutorials/keras/save_and_load)

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10)
    ])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    return model

In [3]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train.shape,x_train.dtype)
#print(tf.math.bincount(x_train[0]))
#print(tf.reduce_sum(tf.math.bincount(x_train[0])))
# input 的值調整成介於0與1之間
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
model.fit(x_train, y_train, epochs=30)

Epoch 1/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2987 - accuracy: 0.9135
Epoch 2/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1429 - accuracy: 0.9583
Epoch 3/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1063 - accuracy: 0.9677
Epoch 4/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0866 - accuracy: 0.9731
Epoch 5/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0738 - accuracy: 0.9769
Epoch 6/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0648 - accuracy: 0.9794
Epoch 7/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0562 - accuracy: 0.9819
Epoch 8/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0524 - accuracy: 0.9829
Epoch 9/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0484 - accuracy: 0.9838
Epoch 10/30
1875/1875 [==============================] - 3s 1ms/step - loss: 0.044

## model.save_weights, model.load_weights

In [6]:
model.save_weights('./checkpoints/beginner2_checkpoint')

In [7]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0867 - accuracy: 0.9813 - 426ms/epoch - 1ms/step


[0.08665207028388977, 0.9812999963760376]

In [8]:
model = create_model()
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 2.3405 - accuracy: 0.0996 - 440ms/epoch - 1ms/step


[2.340461015701294, 0.09960000216960907]

In [9]:
model.load_weights('./checkpoints/beginner2_checkpoint')
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0867 - accuracy: 0.9813 - 298ms/epoch - 953us/step


[0.08665207028388977, 0.9812999963760376]

In [20]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [21]:
def predict(gray28s):
    p=probability_model(gray28s)
    print(p)
    return np.argmax(p, axis=-1)

In [22]:
def mypredict(imgname):
    myimg = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(myimg, cv2.COLOR_BGRA2GRAY)
    gray28 = cv2.resize(gray, (28,28), interpolation = cv2.INTER_AREA)
    d = predict(np.array([gray28/255.0]))[0]
    print(imgname, d)

In [24]:
for name in ["b-zero-fat.png","b-one-fat.png","b-two-fat.png","b-three-fat.png","b-four-fat.png","b-five-fat.png",
             "b-six-fat.png","b-seven-fat.png","b-seven-fat-4.png","b-eight-fat.png","b-nine-fat.png"]:
    mypredict(name)

tf.Tensor(
[[1.0000000e+00 6.3833978e-19 2.3271632e-08 2.6423198e-14 2.4708245e-31
  3.1036513e-16 7.6402338e-15 4.1588187e-13 9.1698947e-25 1.0978578e-15]], shape=(1, 10), dtype=float32)
b-zero-fat.png 0
tf.Tensor(
[[7.5788743e-21 1.0000000e+00 4.1057720e-11 7.4821979e-13 3.2915878e-11
  3.9842136e-14 8.2670530e-11 6.5367593e-11 3.4103209e-08 3.6286227e-14]], shape=(1, 10), dtype=float32)
b-one-fat.png 1
tf.Tensor(
[[1.5474612e-15 1.5063457e-06 9.9999690e-01 1.5619333e-06 5.2657149e-31
  2.8118317e-17 9.7544179e-26 2.2829725e-15 3.3958891e-11 6.2667489e-18]], shape=(1, 10), dtype=float32)
b-two-fat.png 2
tf.Tensor(
[[4.4580439e-28 2.0274766e-13 7.4609185e-10 1.0000000e+00 9.1779853e-31
  8.1636864e-10 2.6625814e-33 2.9667218e-19 4.5340381e-13 8.7716954e-18]], shape=(1, 10), dtype=float32)
b-three-fat.png 3
tf.Tensor(
[[9.8806021e-14 3.1268575e-25 7.8124835e-08 8.0550279e-14 9.9999988e-01
  2.8183723e-11 2.0920643e-08 9.9575552e-16 8.2954443e-24 5.5189950e-09]], shape=(1, 10), dtype=fl

In [25]:
mypredict("b-seven-fat-5.png")

tf.Tensor(
[[7.5443054e-08 8.0704223e-07 4.9037926e-02 5.1037267e-02 3.2461307e-17
  3.7588404e-07 8.0255776e-18 8.9564675e-01 4.1999338e-05 4.2348043e-03]], shape=(1, 10), dtype=float32)
b-seven-fat-5.png 7


In [26]:
mypredict("b-nine-fat-2.png")

tf.Tensor(
[[1.52254769e-13 4.17352858e-04 2.27312703e-05 9.54105258e-01
  1.28128566e-08 2.06586779e-11 3.01294207e-08 4.54511233e-02
  1.93705068e-06 1.59269985e-06]], shape=(1, 10), dtype=float32)
b-nine-fat-2.png 3


In [27]:
mypredict("b-nine-fat-3.png")

tf.Tensor(
[[7.2808967e-14 3.8482364e-16 1.4957824e-09 3.8481790e-08 6.8865722e-09
  1.2202889e-09 1.4048791e-16 1.0853518e-03 6.5921868e-10 9.9891460e-01]], shape=(1, 10), dtype=float32)
b-nine-fat-3.png 9


In [29]:
mypredict("b-nine-fat-5.png")

tf.Tensor(
[[2.29579072e-21 2.76885472e-33 7.64476186e-18 3.49862805e-09
  8.61750907e-15 1.05085993e-08 1.04361139e-19 1.00657625e-11
  1.09710186e-06 9.99998927e-01]], shape=(1, 10), dtype=float32)
b-nine-fat-5.png 9


In [31]:
mypredict("b-three-5w.jpg")

tf.Tensor(
[[2.5210991e-15 1.3499187e-18 3.8782567e-02 9.6110475e-01 1.4307099e-37
  1.1246540e-04 7.7302298e-26 1.8732242e-20 6.4795138e-08 1.1882284e-15]], shape=(1, 10), dtype=float32)
b-three-5w.jpg 3


## 用 draw.html 可以製作數字檔案，複製貼上小畫家比直接存檔好
draw.html 從[這個gist](https://gist.github.com/bencentra/91350fe91c377c1ca574)修改而得。